In [4]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon, Rectangle
from PIL import Image
import ast
import numpy as np
import cv2
from shapely.geometry import Polygon as ShapelyPolygon
from shapely.geometry import Point, LineString
from math import radians, cos, sin
import os

In [5]:
# Function to reverse coordinates diagonally
def reverse_diagonal(coords):
    return [(coord[1], coord[0]) for coord in coords]

# Function to check if a rectangle intersects with any obstacle polygon
def rectangle_intersects_obstacle(x, y, width, height, obstacle_polygons):
    rect_coords = [(x, y), (x + width, y), (x + width, y + height), (x, y + height)]
    rect = ShapelyPolygon(rect_coords)
    for obstacle_poly in obstacle_polygons:
        if rect.intersects(obstacle_poly):
            return True
    return False

# Function to find the contour of a polygon using cv2
def find_polygon_contour(mask_array_reversed, img_shape):
    mask = np.zeros(img_shape[:2], dtype=np.uint8)
    polygon_points = np.array(mask_array_reversed, dtype=np.int32)
    cv2.fillPoly(mask, [polygon_points], 255)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour = max(contours, key=cv2.contourArea)
    contour_coords = [(point[0][0], point[0][1]) for point in contour]
    return contour_coords

# Function to approximate a polygon contour
def approximate_polygon(contour, tolerance):
    epsilon = tolerance * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)
    approx = np.squeeze(approx)
    return approx.tolist()

# Function to check if a point is inside a polygon using shapely
def point_inside_polygon(x, y, poly_coords):
    shapely_poly = ShapelyPolygon(poly_coords)
    point = Point(x, y)
    return shapely_poly.contains(point)

# Function to check if a rectangle intersects with any line in a list of lines
def rectangle_intersects_lines(x, y, width, height, lines):
    rect = ShapelyPolygon([(x, y), (x + width, y), (x + width, y + height), (x, y + height)])
    for line in lines:
        if rect.intersects(line):
            return True
        for i in range(4):
            point = Point(rect.exterior.coords[i])
            if line.contains(point):
                return True
    return False


In [6]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon, Rectangle
from PIL import Image
import ast
import numpy as np
import cv2
from shapely.geometry import Polygon as ShapelyPolygon
from shapely.geometry import Point, LineString
from math import radians, cos, sin
import os

# Constants
pv_module_width = 31.42  # Width of PV module in pixels
pv_module_height = 15.64  # Height of PV module in pixels
pv_module_spacing = 10   # Spacing between PV modules in pixels
tilt_angle_degrees = 30  # Tilt angle for PV modules in degrees

# Paths
csv_path = '/pv/testcsv.csv'
image_folder = '/Desktop/pv/'
output_folder = '/pv/outputs/'

# Read the CSV file
df = pd.read_csv(csv_path)

# Ensure the 'Mask Coordinates' column is clean and parsed as lists of tuples
df['Mask Coordinates'] = df['Mask Coordinates'].apply(lambda x: ast.literal_eval(x.replace('[', '').replace(']', '')))
df['PV_Number'] = 0

# Function to reverse coordinates diagonally
def reverse_diagonal(coords):
    return [(coord[1], coord[0]) for coord in coords]

# Function to check if a rectangle intersects with any obstacle polygon
def rectangle_intersects_obstacle(x, y, width, height, obstacle_polygons):
    rect_coords = [(x, y), (x + width, y), (x + width, y + height), (x, y + height)]
    rect = ShapelyPolygon(rect_coords)
    for obstacle_poly in obstacle_polygons:
        if rect.intersects(obstacle_poly):
            return True
    return False

# Function to find the contour of a polygon using cv2
def find_polygon_contour(mask_array_reversed, img_shape):
    mask = np.zeros(img_shape[:2], dtype=np.uint8)
    polygon_points = np.array(mask_array_reversed, dtype=np.int32)
    cv2.fillPoly(mask, [polygon_points], 255)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour = max(contours, key=cv2.contourArea)
    contour_coords = [(point[0][0], point[0][1]) for point in contour]
    return contour_coords

# Function to approximate a polygon contour
def approximate_polygon(contour, tolerance):
    epsilon = tolerance * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)
    approx = np.squeeze(approx)
    return approx.tolist()

# Function to check if a point is inside a polygon using shapely
def point_inside_polygon(x, y, poly_coords):
    shapely_poly = ShapelyPolygon(poly_coords)
    point = Point(x, y)
    return shapely_poly.contains(point)

# Function to check if a rectangle intersects with any line in a list of lines
def rectangle_intersects_lines(x, y, width, height, lines):
    rect = ShapelyPolygon([(x, y), (x + width, y), (x + width, y + height), (x, y + height)])
    for line in lines:
        if rect.intersects(line):
            return True
        for i in range(4):
            point = Point(rect.exterior.coords[i])
            if line.contains(point):
                return True
    return False

# Process each image listed in the CSV file
for image_name in df['Image Name'].unique():
    df_filtered = df[df['Image Name'] == image_name]

    image_path = os.path.join(image_folder, image_name)
    output_image_path = os.path.join(output_folder, f"{os.path.splitext(image_name)[0]}_pv.jpg")
    
    image = Image.open(image_path)
    image_np = np.array(image)
    fig, ax = plt.subplots(1)
    ax.imshow(image_np)

    obstacle_polygons = []

    # Iterate over each obstacle polygon in the image
    for idx, row in df_filtered[df_filtered['Class Name'] == 'obstacle'].iterrows():
        mask_coords = row['Mask Coordinates']
        mask_array = [(mask_coords[i], mask_coords[i+1]) for i in range(0, len(mask_coords), 2)]
        mask_array_reversed = reverse_diagonal(mask_array)
        img_shape = image.size[::-1] + (3,)
        obstacle_contour_coords = find_polygon_contour(mask_array_reversed, img_shape)
        
        # Approximate contour to polygon with a tolerance (adjust as needed)
        tolerance = 0.009  # Example tolerance, adjust based on your needs
        obstacle_contour_np = np.array(obstacle_contour_coords, dtype=np.int32)
        
        # Approximate contour to polygon
        obstacle_polygon = approximate_polygon(obstacle_contour_np, tolerance)
        
        # Create a shapely polygon from the approximated polygon
        shapely_polygon = ShapelyPolygon(obstacle_polygon)
        
        # Draw the obstacle polygon shape
        polygon_contour = Polygon(obstacle_polygon, closed=True, edgecolor='b', facecolor='none', linewidth=1, zorder=2)
        ax.add_patch(polygon_contour)
        
        # Store obstacle polygons for later use
        obstacle_polygons.append(shapely_polygon)
    
    for idx, row in df_filtered.iterrows():
        mask_coords = row['Mask Coordinates']
        mask_array = [(mask_coords[i], mask_coords[i+1]) for i in range(0, len(mask_coords), 2)]
        mask_array_reversed = reverse_diagonal(mask_array)
        img_shape = image.size[::-1] + (3,)
        contour_coords = find_polygon_contour(mask_array_reversed, img_shape)
        shapely_polygon = ShapelyPolygon(contour_coords)
        polygon_contour = Polygon(contour_coords, closed=True, edgecolor='r', facecolor='none', linewidth=1, zorder=2)
        ax.add_patch(polygon_contour)
        class_name = row['Class Name']
        pv_rectangles_to_draw = []

        if class_name == 'sloped_roof':
            sloped_roof_pv_count = 0
            # Find and draw the contour of the polygon
            contour_coords = find_polygon_contour(mask_array_reversed, img_shape)
            
            # Create a shapely polygon from contour coordinates
            shapely_polygon = ShapelyPolygon(contour_coords)
            
            # Find the minimum bounding rectangle around the contour
            rect = cv2.minAreaRect(np.array(contour_coords))

            # Get the vertices of the rectangle
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            
            # Convert the contour to a list of (x, y) tuples
            contour_coords = [(point[0], point[1]) for point in box]

            # Create a shapely polygon from the rectangle coordinates
            shapely_polygon = ShapelyPolygon(contour_coords)

            # Draw the rectangle contour on the plot
            polygon_contour = Polygon(contour_coords, closed=True, edgecolor='r', facecolor='none', linewidth=1, zorder=2)
            ax.add_patch(polygon_contour)
            # Calculate the minimum bounding rectangle around the contour
            rect = cv2.minAreaRect(np.array(contour_coords))
            
            # Get the vertices of the rectangle
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            
            # Convert the contour to a list of (x, y) tuples
            contour_coords = [(point[0], point[1]) for point in box]
            
            # Calculate keypoints inside the rectangle bounding box
            p1 = np.array(contour_coords[0])  # Bottom-left corner
            p2 = np.array(contour_coords[1])  # Bottom-right corner
            p3 = np.array(contour_coords[2])  # Top-right corner
            p4 = np.array(contour_coords[3])  # Top-left corner
            
            # Calculate midpoint along the shorter sides of the rectangle (width)
            if np.linalg.norm(p1 - p2) < np.linalg.norm(p2 - p3):
                width_midpoint1 = p3
                width_midpoint2 = p4
                orientation_line = [p1, p2]
            else:
                width_midpoint1 = p2
                width_midpoint2 = p3
                orientation_line = [p2, p3]
            
            # Calculate the center of the rectangle
            center_x = (p1[0] + p3[0]) // 2
            center_y = (p1[1] + p3[1]) // 2
            
            # Calculate the second line of symmetry (perpendicular to the first line)
            if orientation_line[0][0] == orientation_line[1][0]:  # Vertical line case
                second_line = [(center_x, p1[1]), (center_x, p3[1])]
            else:
                slope = (orientation_line[1][1] - orientation_line[0][1]) / (orientation_line[1][0] - orientation_line[0][0])
                intercept = center_y - slope * center_x
                x1 = center_x - 100  # Extend the line beyond the rectangle for plotting
                x2 = center_x + 100
                y1 = int(slope * x1 + intercept)
                y2 = int(slope * x2 + intercept)
                second_line = [(x1, y1), (x2, y2)]
            
            # Calculate the perpendicular line to the second line of symmetry (rotated by 90 degrees)
            if second_line[0][0] == second_line[1][0]:  # Vertical line case
                perpendicular_line = [(center_x , center_y), (center_x , center_y)]
            else:
                height = np.linalg.norm(p1 - p3)  # height of the rectangle
                delta = int(0.2 * height)  # 30% of the rectangle height
                slope_perpendicular = -1 / slope  # Slope perpendicular to the second line
                intercept_perpendicular = center_y - slope_perpendicular * center_x
                x1_perpendicular = center_x - delta
                x2_perpendicular = center_x + delta
                y1_perpendicular = int(slope_perpendicular * x1_perpendicular + intercept_perpendicular)
                y2_perpendicular = int(slope_perpendicular * x2_perpendicular + intercept_perpendicular)
                perpendicular_line = [(x1_perpendicular, y1_perpendicular), (x2_perpendicular, y2_perpendicular)]
            ax.plot([perpendicular_line[0][0], perpendicular_line[1][0]], [perpendicular_line[0][1], perpendicular_line[1][1]], color='blue', linestyle='-', linewidth=1, zorder=5)
            # Draw the perpendicular line to the second line of symmetry (rotated by 90 degrees)
            ax.plot([perpendicular_line[0][0], perpendicular_line[1][0]], [perpendicular_line[0][1], perpendicular_line[1][1]], color='blue', linestyle='-', linewidth=1, zorder=5)
            
            # Define the vertices of the two triangles
            triangle1 = [orientation_line[0], orientation_line[1], perpendicular_line[0]]
            triangle2 = [width_midpoint1, width_midpoint2, perpendicular_line[1]]
            
            # Draw the two triangles
            triangle1_patch = Polygon(triangle1, closed=True, edgecolor='blue', facecolor='none', linewidth=1, zorder=5)
            triangle2_patch = Polygon(triangle2, closed=True, edgecolor='blue', facecolor='none', linewidth=1, zorder=5)
            ax.add_patch(triangle1_patch)
            ax.add_patch(triangle2_patch)
            
            # Convert lines and triangles to shapely objects
            orientation_line_shapely = LineString(orientation_line)
            perpendicular_line_shapely = LineString(perpendicular_line)
            triangle1_shapely = ShapelyPolygon(triangle1)
            triangle2_shapely = ShapelyPolygon(triangle2)
            
            # Iterate over the width and height of the polygon to place PV modules
            min_x = min(coord[0] for coord in contour_coords)
            max_x = max(coord[0] for coord in contour_coords)
            min_y = min(coord[1] for coord in contour_coords)
            max_y = max(coord[1] for coord in contour_coords)
            
            # Place PV modules inside the overall polygon (shapely_polygon)
            for x in range(int(min_x), int(max_x), int(pv_module_height + pv_module_spacing)):
                for y in range(int(min_y), int(max_y), int(pv_module_width + pv_module_spacing)):
                    center_x = x + pv_module_height // 2
                    center_y = y + pv_module_width // 2
                    if (point_inside_polygon(center_x, center_y, shapely_polygon) and
                        not rectangle_intersects_lines(x, y, pv_module_height, pv_module_width, 
                            [orientation_line_shapely, perpendicular_line_shapely, triangle1_shapely, triangle2_shapely]) and
                            not rectangle_intersects_obstacle(x, y, pv_module_width, pv_module_height, obstacle_polygons)):
                        
                        pv_rectangles_to_draw.append((x, y))
                        sloped_roof_pv_count += 1  
               
            # Place PV modules inside the triangles (triangle1_shapely and triangle2_shapely)
            for x in range(int(min_x), int(max_x), int(pv_module_height + pv_module_spacing)):
                for y in range(int(min_y), int(max_y), int(pv_module_width + pv_module_spacing)):
                    center_x = x + pv_module_height / 2
                    center_y = y + pv_module_width / 2
                    if (point_inside_polygon(center_x, center_y, shapely_polygon) and
                        (triangle1_shapely.contains(Point(center_x, center_y)) or
                        triangle2_shapely.contains(Point(center_x, center_y))) and
                        not rectangle_intersects_lines(x, y, pv_module_height, pv_module_width, 
                            [orientation_line_shapely, perpendicular_line_shapely, triangle1_shapely.boundary, triangle2_shapely.boundary]) and
                            not rectangle_intersects_obstacle(x, y, pv_module_width, pv_module_height, obstacle_polygons)):
                        

                        pv_rectangles_to_draw.append((x, y))
                        sloped_roof_pv_count += 1  

            
        elif class_name == 'flat_roof':
            flat_roof_pv_count = 0 

            # Approximate contour to polygon with a tolerance (adjust as needed)
            tolerance = 0.009  # Example tolerance, adjust based on your needs
            contour_np = np.array(contour_coords, dtype=np.int32)
            
            # Approximate contour to polygon
            flat_polygon = approximate_polygon(contour_np, tolerance)
            
            # Create a shapely polygon from the approximated polygon
            shapely_polygon = ShapelyPolygon(flat_polygon)
            
            # Draw the polygon shape
            polygon_contour = Polygon(flat_polygon, closed=True, edgecolor='b', facecolor='none', linewidth=1, zorder=2)
            ax.add_patch(polygon_contour)
            
            # Define the orientation line for flat roofs (horizontal line)
            min_x = min(coord[0] for coord in contour_coords)
            max_x = max(coord[0] for coord in contour_coords)
            orientation_line_shapely_flat = LineString([(min_x, 0), (max_x, 0)])  # Horizontal line along the x-axis
            
            
            # Iterate over the width and height of the polygon to place PV modules
            min_x = min(coord[0] for coord in contour_coords)
            max_x = max(coord[0] for coord in contour_coords)
            min_y = min(coord[1] for coord in contour_coords)
            max_y = max(coord[1] for coord in contour_coords)
            
            # Place PV modules inside the overall polygon (shapely_polygon)
            for x in range(int(min_x), int(max_x), int(pv_module_height + pv_module_spacing)):
                for y in range(int(min_y), int(max_y), int(pv_module_width + pv_module_spacing)):
                    if point_inside_polygon(x + pv_module_width / 2, y + pv_module_height / 2, contour_coords):

                        pv_rectangles_to_draw.append((x, y))   
                        flat_roof_pv_count += 1  

        elif class_name == 'obstacle':
            continue 


        # Iterate over PV rectangles to draw
        for pv_rect in pv_rectangles_to_draw:
            x, y = pv_rect  # Unpack tuple
            width = pv_module_width
            height = pv_module_height

            # Check if PV module intersects with any obstacle polygons
                # Draw PV module rectangle
                # Draw PV module rotated 90 degrees
            rect_pv = Rectangle((x, y), pv_module_height, pv_module_width, edgecolor='g', facecolor='none', linewidth=1, zorder=10)
            ax.add_patch(rect_pv)




    # Update PV_Number column for this image in the DataFrame
    for idx, row in df_filtered.iterrows():
        if row['Class Name'] == 'sloped_roof':
            df.loc[idx, 'PV_Number'] = sloped_roof_pv_count
        elif row['Class Name'] == 'flat_roof':
            df.loc[idx, 'PV_Number'] = flat_roof_pv_count



    plt.axis('off')
    plt.savefig(output_image_path, bbox_inches='tight', pad_inches=0, dpi=300)
    plt.close(fig)
# Save updated DataFrame to CSV
df.to_csv(csv_path, index=False)
print('Processing complete. Output images saved in:', output_folder)


Processing complete. Output images saved in: /Users/insafbenlamari/Desktop/pv/outputs/
